# Magic Square Game
This game is defined in the following blog: https://www.quantamagazine.org/the-universes-ultimate-complexity-revealed-by-simple-quantum-games-20190305/

This project simulates th magic square game and discovers an optimal strategy to win the game with the highest probability.

In [1]:
import numpy as np
import random

In [2]:
# define 3x3 grid
def empty_grid(rows,cols):
    grid = np.zeros((rows,cols))
    grid.fill(np.nan)
    return grid

# all possible row-column assignments (pairs)
def get_asgns(grid):
    grid_rows, grid_cols = grid.shape
    grid_asgns = set()
    for r in range(grid_rows):
        for c in range(grid_cols):
            grid_asgns.add((r,c,))
    
    return grid_asgns

# The referee will assign alice and bob each a row and column respectively.
# We make sure a previous selection is not chosen again so that our games
# will at most last 5 rounds before a contradiction is reached. 3 is the  
# fewest number of rounds required.
def assign(grid, prev_asgn = None):
    if prev_asgn == None:
        prev_asgn = set()
    grid_asgns = get_asgns(grid)
    asgn_opts = grid_asgns.difference(prev_asgn)
    # randomly choose a unique assignment
    alice_row_asgn, bob_col_asgn = random.sample(asgn_opts,1)[0]
    
    return (alice_row_asgn, bob_col_asgn)

# get all possible even or odd fill combos with n cols or rows
def per(n, even=True):
    pats = []
    for i in range(1<<n):
        s=bin(i)[2:]
        s='0'*(n-len(s))+s
        s= list(map(int,list(s)))
        if even:
            if(sum(s) % 2 == 0):
                pats = pats + [s] 
        else:
            if(sum(s) % 2 == 1):
                pats = pats + [s] 
        
    return pats

# given a row or column vector, find the combos that wont change the currently filled in values
def remaining(opts, vector, ind):
#     print(f"opts: {opts}")
#     print(f"vector: {vector}")
#     print(f"ind: {ind}")
    rem = set()
    vector[ind] 
    for o in opts:
        o = np.array(o)
#         print(o[ind])
#         print(vector[ind])
        if all(o[ind] == vector[ind]):
            rem.add(tuple(o))
#     print(rem)
#     print()
    return rem

# given a partially filled grid, execute another round of alice and bob filling in their assigned row and column
def fill_vectors(grid, asgn):
    end = False
    
    # get selected row and column from grid
    row_asgn, col_asgn = asgn
    asgnd_row = grid[row_asgn]
    asgnd_col = grid[:,col_asgn]
    
    # get all possible row and column options for odds and evens
    row_opts, col_opts = per(grid.shape[0], False), per(grid.shape[1], True)
    
    
    
    # check if assigned row and column are properly even and odd, otherwise a requirement is broken and we end the game
    # if there are no nans and if the row is not an odd option at all
    if not any(np.isnan(asgnd_row)) and not (list(asgnd_row) in row_opts):
        end = True
#         print(f"Requirement not met for row {asgnd_row}! You successfully completed {score(grid)} row and columns")
        return (grid, end, score(grid))
    # if there are no nans and if the col is not an even option at all
    if not any(np.isnan(asgnd_col)) and not (list(asgnd_col) in col_opts):
        end = True
#         print(f"Requirement not met for column {asgnd_col}! You successfully completed {score(grid)} row and columns")
        return (grid, end, score(grid))
    
    
    
    # find where there are values in those vectors we need to consider when choosing new option to fill so we don't
    # overwrite our grid state with completely new values
    where_row_val = np.where(~np.isnan(asgnd_row))[0]
    where_col_val = np.where(~np.isnan(asgnd_col))[0]
    
    # find remaining options to choose from to fill vector
    rem_row_opts, rem_col_opts = remaining(row_opts,asgnd_row,where_row_val), remaining(col_opts,asgnd_col,where_col_val)
    alice_row, bob_col = random.sample(rem_row_opts,1)[0], random.sample(rem_col_opts,1)[0]
    # if alice and bobs intersecting columns clash, then the game is over!
    end = alice_row[col_asgn] != bob_col[row_asgn]
    if end:
#         print(f"Conflict! You successfully completed {score(grid)} row and columns")
        return (grid, end, score(grid))
    else:
        grid[row_asgn] = alice_row
        grid[:,col_asgn] = bob_col
        
        return (grid, end, score(grid))

def score(grid):
    reqs_met = empty_grid(grid.shape[0],grid.shape[1])
    # reqs_met
    for r in range(grid.shape[0]):
        for c in range(grid.shape[1]):
            if not any(np.isnan(np.append(grid[r],grid[:,c]))) and (sum(grid[r]) % 2 == 1 and sum(grid[:,c]) % 2 == 0):
                reqs_met[r,c] = True
            else:
                reqs_met[r,c] = False
    reqs_met
    
    correct = 0
    for r,c in zip(range(grid.shape[0]), range(grid.shape[1])):
        if 1 in reqs_met[r]:
            correct += 1
        if 1 in reqs_met[:,c]:
            correct += 1
    
    total = grid.shape[0] + grid.shape[1]
    
    return f"{correct}/{total}"

In [3]:
strategies = []

for i in range(1000):
    grid = empty_grid(3,3)
    prev_asgn = set()
    strategy = [grid.copy()]
    
    while True:    
        asgnmt = assign(grid,prev_asgn)
        prev_asgn.add(asgnmt)
        new_grid, end, correct = fill_vectors(grid,asgnmt)
        if end:
#             print("\nGoal: odd rows, even columns")
#             print(new_grid)
            
            if correct == "5/6":
                strategies = strategies + [strategy]
            break
        else:
            grid = new_grid
            strategy = strategy + [grid.copy()]

In [4]:
# does contain duplicates
len(strategies)

261

In [5]:
for i in range(len(strategies)):
    print(f"Strategy {i+1}")
    for j in range(len(strategies[i])):
        print("step " + str(j))
        print(strategies[i][j])
        print()
    print()

Strategy 1
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[nan  0. nan]
 [ 1.  0.  0.]
 [nan  0. nan]]

step 2
[[nan  0. nan]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]

step 3
[[nan  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]

step 4
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


Strategy 2
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[ 1.  0.  0.]
 [nan  0. nan]
 [nan  0. nan]]

step 2
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [nan  0. nan]]

step 3
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]

step 4
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


Strategy 3
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[ 0.  1.  0.]
 [nan nan  0.]
 [nan nan  0.]]

step 2
[[ 0.  1.  0.]
 [nan  0.  0.]
 [nan  1.  0.]]

step 3
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [nan  1.  0.]]

step 4
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]

step 5
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


Strategy 4
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[ 1.  0.  0.]
 [nan nan  0.]
 [nan nan  0.]]

step 2
[[ 1.  0.  0.]
 [ 1

step 3
[[ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 1. nan  1.]]

step 4
[[1. 0. 0.]
 [0. 0. 1.]
 [1. 1. 1.]]


Strategy 66
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[nan  0. nan]
 [nan  1. nan]
 [ 0.  1.  0.]]

step 2
[[ 1.  0.  0.]
 [nan  1.  0.]
 [ 0.  1.  0.]]

step 3
[[ 1.  0.  0.]
 [nan  1.  0.]
 [ 0.  1.  0.]]

step 4
[[ 1.  0.  0.]
 [nan  1.  0.]
 [ 0.  1.  0.]]

step 5
[[1. 0. 0.]
 [1. 1. 0.]
 [0. 1. 0.]]


Strategy 67
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[ 1.  1.  1.]
 [nan  0. nan]
 [nan  1. nan]]

step 2
[[ 1.  1.  1.]
 [ 1.  0.  0.]
 [nan  1.  1.]]

step 3
[[1. 1. 1.]
 [1. 0. 0.]
 [0. 1. 1.]]

step 4
[[1. 1. 1.]
 [1. 0. 0.]
 [0. 1. 1.]]


Strategy 68
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[nan nan  0.]
 [ 1.  0.  0.]
 [nan nan  0.]]

step 2
[[ 1. nan  0.]
 [ 1.  0.  0.]
 [ 0. nan  0.]]

step 3
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]


Strategy 69
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[ 0. nan nan]

step 4
[[1. 1. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]


Strategy 138
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[ 1.  1.  1.]
 [nan  0. nan]
 [nan  1. nan]]

step 2
[[ 1.  1.  1.]
 [ 1.  0.  0.]
 [nan  1. nan]]

step 3
[[1. 1. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]

step 4
[[1. 1. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]

step 5
[[1. 1. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]

step 6
[[1. 1. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]


Strategy 139
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[nan  0. nan]
 [nan  1. nan]
 [ 0.  1.  0.]]

step 2
[[nan  0.  0.]
 [nan  1.  0.]
 [ 0.  1.  0.]]

step 3
[[ 1.  0.  0.]
 [nan  1.  0.]
 [ 0.  1.  0.]]

step 4
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


Strategy 140
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[nan nan  0.]
 [nan nan  1.]
 [ 0.  0.  1.]]

step 2
[[ 0. nan  0.]
 [ 0. nan  1.]
 [ 0.  0.  1.]]

step 3
[[ 0. nan  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]

step 4
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]

step 5
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


Str

 [ 1.  1.  1.]]

step 3
[[1. 0. 0.]
 [0. 1. 1.]
 [1. 1. 1.]]

step 4
[[1. 0. 0.]
 [0. 1. 1.]
 [1. 1. 1.]]


Strategy 208
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[ 0. nan nan]
 [ 0. nan nan]
 [ 0.  1.  0.]]

step 2
[[ 0.  0.  1.]
 [ 0. nan nan]
 [ 0.  1.  0.]]

step 3
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


Strategy 209
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[nan nan  1.]
 [nan nan  0.]
 [ 0.  0.  1.]]

step 2
[[nan nan  1.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]

step 3
[[nan  1.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]

step 4
[[0. 1. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]

step 5
[[0. 1. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]

step 6
[[0. 1. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]


Strategy 210
step 0
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

step 1
[[nan  0. nan]
 [nan  1. nan]
 [ 1.  1.  1.]]

step 2
[[nan  0.  0.]
 [nan  1.  1.]
 [ 1.  1.  1.]]

step 3
[[nan  0.  0.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]

step 4
[[nan  0.  0.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]

step 5
[[0. 0. 0.]
 [1.